In [1]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from utils import load_data, accuracy
from models import GCNWithKAN


In [2]:

# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=500,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=1,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

args = parser.parse_known_args()[0]
args.cuda = not args.no_cuda and torch.cuda.is_available()

np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# Load data
# adj, features,
adj, features, labels, idx_train, idx_val, idx_test = load_data()

# Model and optimizer
model = GCNWithKAN(nfeat=features.shape[1],
            nhid=args.hidden,
            nclass=labels.max().item() + 1,
            dropout=args.dropout)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

if args.cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))


# Train model
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()


Loading cora dataset...


/root/pygcn/pygcn/utils.py:80: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /opt/conda/conda-bld/pytorch_1724789220573/work/torch/csrc/utils/tensor_new.cpp:641.)
  return torch.sparse.FloatTensor(indices, values, shape)


Epoch: 0001 loss_train: 1.9815 acc_train: 0.1500 loss_val: 1.9307 acc_val: 0.1567 time: 0.3779s
Epoch: 0002 loss_train: 1.9196 acc_train: 0.1929 loss_val: 1.8803 acc_val: 0.1567 time: 0.0121s
Epoch: 0003 loss_train: 1.8776 acc_train: 0.1857 loss_val: 1.8471 acc_val: 0.1567 time: 0.0117s
Epoch: 0004 loss_train: 1.8566 acc_train: 0.1929 loss_val: 1.8260 acc_val: 0.3500 time: 0.0116s
Epoch: 0005 loss_train: 1.8605 acc_train: 0.2643 loss_val: 1.8168 acc_val: 0.3500 time: 0.0115s
Epoch: 0006 loss_train: 1.8490 acc_train: 0.2857 loss_val: 1.8235 acc_val: 0.3500 time: 0.0115s
Epoch: 0007 loss_train: 1.8402 acc_train: 0.2786 loss_val: 1.8308 acc_val: 0.3500 time: 0.0115s
Epoch: 0008 loss_train: 1.8394 acc_train: 0.2857 loss_val: 1.8143 acc_val: 0.3500 time: 0.0115s
Epoch: 0009 loss_train: 1.8184 acc_train: 0.2786 loss_val: 1.8032 acc_val: 0.3500 time: 0.0115s
Epoch: 0010 loss_train: 1.8473 acc_train: 0.2786 loss_val: 1.8024 acc_val: 0.3500 time: 0.0115s
Epoch: 0011 loss_train: 1.8036 acc_train